In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Project').getOrCreate()

In [2]:
df = spark.read.csv('/FileStore/tables/train.csv', inferSchema=True, header = True)

In [3]:
df.show()

In [4]:
df.count()

In [5]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

In [6]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [7]:
#So the columns containing missing values are: Gender, Married, Dependents, Self-Employed, LoanAmount, Loan_Amount_Term and Credit _History
#There are various ways of imputing missing values: here we will replace missing values with mean for continuous values and mode for categorical. In the later part, we will run boosting and random forest without imputing the missing values as they have capacity to treat it themselves and then test the accuracy.

In [8]:
df.printSchema()

In [9]:
from pyspark.sql.functions import mean

In [10]:
mean_value = df.select(mean(df['LoanAmount'])).collect()

In [11]:
mean_loan_amount = mean_value[0][0]

In [12]:
df = df.na.fill(mean_loan_amount, subset = ['LoanAmount'])

In [13]:
df.show()

In [14]:
import matplotlib.pyplot as plt


In [15]:
df.createOrReplaceTempView("loan")

In [16]:
spark.sql("SELECT * FROM loan").show()

In [17]:
results = spark.sql("select LoanAmount from loan where LoanAmount is not null")
results.collect()

In [18]:
result_array = results.rdd.map(lambda row : row.LoanAmount).collect()
result_array

In [19]:
%matplotlib inline

In [21]:
fig, ax = plt.subplots()
plt.boxplot(result_array)
ax.set_title('Variance in Loan Amount')

In [22]:
display(fig)

In [23]:
results_lat = spark.sql("select Loan_Amount_Term from loan where Loan_Amount_Term is not null")
result_array_lat = results_lat.rdd.map(lambda row : row.Loan_Amount_Term).collect()

In [24]:
fig, ax = plt.subplots()
plt.boxplot(result_array_lat)
ax.set_title('Variance in Loan Amount Term')

In [25]:
display(fig)

In [26]:
results_ai = spark.sql("select ApplicantIncome from loan where ApplicantIncome is not null")
result_array_ai = results_ai.rdd.map(lambda row : row.ApplicantIncome).collect()

In [27]:
fig, ax = plt.subplots()
plt.boxplot(result_array_ai)
ax.set_title('Variance in Applicant Income')

In [28]:
display(fig)

In [29]:
results_cai = spark.sql("select CoapplicantIncome from loan where CoapplicantIncome is not null")
result_array_cai = results_cai.rdd.map(lambda row : row.CoapplicantIncome).collect()

In [30]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax=fig.add_subplot(111, projection = '3d')

ax.scatter( result_array, result_array_ai, result_array_cai , c='r', marker = 'o')

ax.set_xlabel('LoanAmount')
ax.set_ylabel('ApplicantIncome')
ax.set_zlabel('CoapplicantIncome')


In [31]:
display(fig)

In [32]:
results_gender = spark.sql("select Gender from loan where Gender is not null")
result_array_gender = results_gender.rdd.map(lambda row : row.Gender).collect()

In [33]:
results_ls = spark.sql("select Loan_Status from loan where Loan_Status  is not null")
result_array_ls = results_ls.rdd.map(lambda row : row.Loan_Status ).collect()

In [34]:
results_comb = spark.sql("select ApplicantIncome, Gender, Loan_Status from loan where ApplicantIncome is not null and Gender is not null and  Loan_Status is not null")

In [35]:
results_comb.registerTempTable("combo_table")
display(sqlContext.sql("select * from combo_table"))

In [36]:
%sql select * from combo_table

In [37]:
display(df)